# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.124720e+02     1.576764e+00
 * time: 0.3470571041107178
     1     1.070902e+01     1.057659e+00
 * time: 2.1824259757995605
     2    -1.190309e+01     1.005469e+00
 * time: 2.788555145263672
     3    -3.429769e+01     8.411948e-01
 * time: 3.6946990489959717
     4    -4.794660e+01     6.325187e-01
 * time: 4.554778099060059
     5    -5.706145e+01     1.925421e-01
 * time: 5.50510311126709
     6    -5.990805e+01     1.321651e-01
 * time: 6.137176990509033
     7    -6.096205e+01     5.449740e-02
 * time: 6.743517160415649
     8    -6.134327e+01     8.153182e-02
 * time: 7.349250078201294
     9    -6.163905e+01     3.449308e-02
 * time: 7.932445049285889
    10    -6.183392e+01     2.627582e-02
 * time: 8.551711082458496
    11    -6.198276e+01     1.993348e-02
 * time: 9.171962976455688
    12    -6.204200e+01     1.608759e-02
 * time: 9.768180131912231
    13    -6.210231e+01     1.842067e-02
 * time: 10.361701011657715
    

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671077
    AtomicLocal         -18.8557731
    AtomicNonlocal      14.8522678
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485390 
    Xc                  -19.3336827

    total               -62.261666461854
